In [28]:
#importing programs as needed
import pandas as pd
import re
import os
import csv
import string
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
import spacy

In [29]:
!spacy download en_core_web_trf

2024-01-08 14:11:18.107433: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 14:11:18.107493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 14:11:18.109142: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 14:11:19.569412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.7 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-trf
    Found existing installation: en-core-web-trf 3.6.1
    Uninstalling en-core-web-trf-3.6.1:
      Successfully unins

In [46]:
#creating the corpus of lyrics
#code obtained from github
from bs4 import BeautifulSoup
import re
import lyricsgenius
import requests
from pathlib import Path
import pandas as pd

def clean_up(song_title):
    if "Ft" in song_title:
        before_ft_pattern = re.compile(".*(?=\(Ft)")
        song_title_before_ft = before_ft_pattern.search(song_title).group(0)
        clean_song_title = song_title_before_ft.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    else:
        song_title_no_lyrics = song_title.replace("Lyrics", "")
        clean_song_title = song_title_no_lyrics.strip()
        clean_song_title = clean_song_title.replace("/", "-")
    return clean_song_title

def get_all_songs_from_album(artist, album_name):
    artist = artist.replace(" ", "-")
    album_name = album_name.replace(" ", "-")
    response = requests.get(f"https://genius.com/albums/{artist}/{album_name}")
    html_string = response.text
    document = BeautifulSoup(html_string, "html.parser")
    song_title_tags = document.find_all("h3", attrs={"class": "chart_row-content-title"})
    song_titles = [song_title.text for song_title in song_title_tags]
    clean_songs = []
    for song_title in song_titles:
        clean_song = clean_up(song_title)
        clean_songs.append(clean_song)
    return clean_songs

def download_album_lyrics(artist, album_name):
    # You will need to go to Genius Developers to get your own client access token
    client_access_token = 'insert_your_client_access_token_here'
    LyricsGenius = lyricsgenius.Genius(client_access_token)
    LyricsGenius.remove_section_headers = True
    clean_songs = get_all_songs_from_album(artist, album_name)
    for song in clean_songs:
        song_object = LyricsGenius.search_song(song, artist)
        if song_object != None:
            artist_title = artist.replace(" ", "-")
            album_title = album_name.replace(" ", "-")
            song_title = song.replace("/", "-")
            song_title = song.replace(" ", "-")
            custom_filename=f"{artist_title}_{album_title}/{song_title}"
            Path(f"{artist_title}_{album_title}").mkdir(parents=True, exist_ok=True)
            song_object.save_lyrics(filename=custom_filename, extension='txt', sanitize=False)
        else:
            print('No lyrics')

In [ ]:
#download_album_lyrics("Taylor Swift", "Taylor Swift")
#download_album_lyrics("Taylor Swift", "Fearless")
#download_album_lyrics("Taylor Swift", "Speak Now")
#download_album_lyrics("Taylor Swift", "Red")
#download_album_lyrics("Taylor Swift", "1989")
#download_album_lyrics("Taylor Swift", "Reputation")
#download_album_lyrics("Taylor Swift", "Lover")
#download_album_lyrics("Taylor Swift", "folklore")
#download_album_lyrics("Taylor Swift", "evermore")
#download_album_lyrics("Taylor Swift", "Midnights")

In [31]:
import re
import string
#remove additional text from txt files
def clean_text(text):
    text = text.replace('See Taylor Swift LiveGet tickets as low as $270', ' ')
    text = re.sub('\d*Embed', ' ', text) # remove ending text with number + Embed
    text = re.sub('*Lyrics', ' ', text) # remove ending text with number + Embed
    return text

In [36]:
# Specify the folder names with the lyric data from Genius
directory_paths = [
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_1989",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Fearless",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Lover",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Midnights",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Red",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Reputation",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Speak-Now",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Taylor-Swift",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_evermore",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_folklore"
]

In [33]:
from google.colab import drive
drive.mount('/content/drive')
#in case of error
#drive.mount("/content/drive", force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
def read_text_files(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                song_name = filename.replace('.txt', '')
                lyrics = file.read()
                data.append({"Album": os.path.basename(folder_path), "Song Name": song_name, "Lyrics": lyrics})
    return data

def convert_to_csv(input_folders, output_csv):
    all_data = []
    for folder in input_folders:
        folder_data = read_text_files(folder)
        all_data.extend(folder_data)

    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ["Album", "Song Name", "Lyrics"]
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for row in all_data:
            writer.writerow(row)

input_folders = [
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_1989",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Fearless",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Lover",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Midnights",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Red",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Reputation",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Speak-Now",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_Taylor-Swift",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_evermore",
    "/content/drive/MyDrive/Taylor_Swift/Taylor_Swift_Genius (1)/Taylor-Swift_folklore"
]
output_csv = "Taylor_SwiftCSV.csv"

convert_to_csv(input_folders, output_csv)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#!python -m spacy download en_core_web_trf
#error indicates that there is a version conflict between the installed spacy version (3.7.2) and the version required by the en_core_web_trf model (which requires spacy version less than 3.7.0)
!pip install spacy==3.6.0

import spacy

nlp = spacy.load('en_core_web_trf')
print(nlp.pipe_names)


['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [ ]:
csv_file = "Taylor_SwiftCSV.csv"
df = pd.read_csv(csv_file)

df['Cleaned Lyrics'] = df['Lyrics'].str.replace('\s+', ' ', regex=True).str.strip()
df['Cleaned Lyrics'] = df['Cleaned Lyrics'].str.lower()

df.head()

,Album,Song Name,Lyrics,Cleaned Lyrics
0,Taylor-Swift_1989,Wildest-Dreams,"He said, ""Let's get out of this town\nDrive ou...","he said, ""let's get out of this town drive out..."
1,Taylor-Swift_1989,Welcome-to-New-York,"Walking through a crowd, the village is aglow\...","walking through a crowd, the village is aglow ..."
2,Taylor-Swift_1989,Shake-It-Off,I stay out too late\nGot nothin' in my brain\n...,i stay out too late got nothin' in my brain th...
3,Taylor-Swift_1989,This-Love,Clear blue water\nHigh tide came and brought y...,clear blue water high tide came and brought yo...
4,Taylor-Swift_1989,Bad-Blood,"'Cause baby, now we've got bad blood\nYou know...","'cause baby, now we've got bad blood you know ..."


In [ ]:
#running it again after restarting session
!python -m spacy download en_core_web_trf
!pip install --upgrade spacy
nlp = spacy.load('en_core_web_trf')
print(nlp.pipe_names)

2024-01-08 13:33:34.130587: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 13:33:34.130653: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 13:33:34.133167: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 13:33:36.173709: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.3/460.3 MB 2.5 MB/s eta 0:00:00
  Attempting uninstall: en-core-web-trf
    Found existing installation: en-core-web-trf 3.7.3
    Uninstalling en-core-web-trf-3.7.3:
      Successfully unins

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_trf' (3.6.1) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [37]:
def process_text(text):
    return nlp(text)

In [38]:
df['Doc'] = df['Cleaned Lyrics'].apply(process_text)

In [39]:
def get_token(doc):
    return [(token.text) for token in doc]
df['Tokens'] = df['Doc'].apply(get_token)

In [40]:
tokens = df[['Cleaned Lyrics', 'Tokens']].copy()
tokens.head()

,Cleaned Lyrics,Tokens
0,"he said, ""let's get out of this town drive out...","[he, said, ,, "", let, 's, get, out, of, this, ..."
1,"walking through a crowd, the village is aglow ...","[walking, through, a, crowd, ,, the, village, ..."
2,i stay out too late got nothin' in my brain th...,"[i, stay, out, too, late, got, nothin', in, my..."
3,clear blue water high tide came and brought yo...,"[clear, blue, water, high, tide, came, and, br..."
4,"'cause baby, now we've got bad blood you know ...","['cause, baby, ,, now, we, 've, got, bad, bloo..."


In [41]:
def get_lemma(doc):
    return [(token.lemma_) for token in doc]

df['Lemmas'] = df['Doc'].apply(get_lemma)

In [42]:
def get_pos(doc):
    return [(token.pos_, token.tag_) for token in doc]

df['POS'] = df['Doc'].apply(get_pos)

In [43]:
def extract_named_entities(doc):
    return [ent.label_ for ent in doc.ents]

df['Named_Entities'] = df['Doc'].apply(extract_named_entities)

In [44]:
def extract_ne_words(doc):
    return [ent for ent in doc.ents]

df['NE_Words'] = df['Doc'].apply(extract_ne_words)

In [45]:
df.head()

,Album,Song Name,Lyrics,Cleaned Lyrics,Doc,Tokens,Lemmas,POS,Named_Entities,NE_Words
0,Taylor-Swift_1989,Wildest-Dreams,"He said, ""Let's get out of this town\nDrive ou...","he said, ""let's get out of this town drive out...","(he, said, ,, "", let, 's, get, out, of, this, ...","[he, said, ,, "", let, 's, get, out, of, this, ...","[he, say, ,, "", let, we, get, out, of, this, t...","[(PRON, PRP), (VERB, VBD), (PUNCT, ,), (PUNCT,...","[PERSON, MONEY]","[(taylor, swift), (as, low, as, $)]"
1,Taylor-Swift_1989,Welcome-to-New-York,"Walking through a crowd, the village is aglow\...","walking through a crowd, the village is aglow ...","(walking, through, a, crowd, ,, the, village, ...","[walking, through, a, crowd, ,, the, village, ...","[walk, through, a, crowd, ,, the, village, be,...","[(VERB, VBG), (ADP, IN), (DET, DT), (NOUN, NN)...","[GPE, GPE, GPE, GPE, GPE, GPE, GPE, GPE, GPE, ...","[(new, york), (new, york), (new, york), (new, ..."
2,Taylor-Swift_1989,Shake-It-Off,I stay out too late\nGot nothin' in my brain\n...,i stay out too late got nothin' in my brain th...,"(i, stay, out, too, late, got, nothin', in, my...","[i, stay, out, too, late, got, nothin', in, my...","[I, stay, out, too, late, got, nothin', in, my...","[(PRON, PRP), (VERB, VBP), (ADV, RB), (ADV, RB...","[PERSON, MONEY]","[(taylor, swift), (as, low, as, $)]"
3,Taylor-Swift_1989,This-Love,Clear blue water\nHigh tide came and brought y...,clear blue water high tide came and brought yo...,"(clear, blue, water, high, tide, came, and, br...","[clear, blue, water, high, tide, came, and, br...","[clear, blue, water, high, tide, come, and, br...","[(ADJ, JJ), (ADJ, JJ), (NOUN, NN), (ADJ, JJ), ...","[ORG, MONEY]","[(taylor, swift), (as, low, as, $)]"
4,Taylor-Swift_1989,Bad-Blood,"'Cause baby, now we've got bad blood\nYou know...","'cause baby, now we've got bad blood you know ...","('cause, baby, ,, now, we, 've, got, bad, bloo...","['cause, baby, ,, now, we, 've, got, bad, bloo...","['cause, baby, ,, now, we, 've, get, bad, bloo...","[(SCONJ, IN), (NOUN, NN), (PUNCT, ,), (ADV, RB...","[PERSON, MONEY]","[(taylor, swift), (as, low, as, $)]"


In [46]:
df.to_csv('Taylor_SwiftCSV.csv')